In [108]:
import ipywidgets as widgets
from IPython.display import display

In [109]:
l2_general_organisation_size = widgets.IntText(
      value       = 1000
    , description = 'Organisation Size'
    , disabled    = False
)

In [110]:
l2_box_layout = widgets.Layout(
      display         = 'flex'
    , flex_flow       = 'column'
    , align_content   = 'center'
    , justify_content = 'center'
)

l2_employment_type_mix        = widgets.Box(layout = l2_box_layout)
l2_employment_type_mix_header = widgets.Box(layout = l2_box_layout)
l2_employment_type_mix_body   = widgets.Box(layout = l2_box_layout)

l2_employment_type_mix_values = {
    'permanent full time': 50
  , 'permanent part time': 10
  , 'temporary full time': 20
  , 'temporary part time':  5
  , 'casual'             :  5
  , 'contractor'         : 10
  , 'volunteer'          :  0
}

l2_employment_type_mix_title = widgets.Label(
    value       = 'Emploment Type Mix (percentage)'
  , layout      = {'display':"flex", 'justify_content':"center", 'width':'25%'}
)

l2_employment_type_mix_user_message = widgets.Label(
    value       = ''
  , layout      = {'display':"flex", 'justify_content':"center", 'width':'25%'}
)

l2_employment_type_mix_total = widgets.IntText(
    description = 'Total'
  , value       = 100
  , disabled    = True
  , layout      = {'width': 'max_content'}
  , style       = {'description_width': '50%'}
)

l2_employment_type_mix_header.children = [l2_employment_type_mix_title] + [l2_employment_type_mix_user_message] + [l2_employment_type_mix_total]

l2_employment_type_mix_body.children = [widgets.BoundedIntText(
      description = key
    , value       = value
    , min         = 0
    , max         = 100
    , step        = 1
    , disabled    = False
    , layout      = {'width': 'max_content'}
    , style       = {'description_width': '50%'}
) for i, (key, value) in enumerate(l2_employment_type_mix_values.items())]

def l2_employment_type_mix_on_value_change(change):
  if change['old'] != change['new']:
    l2_employment_type_mix_total.value += change['new'] - change['old']

  if l2_employment_type_mix_total.value == 100:
    l2_employment_type_mix_user_message.value = 'Employment types total 100% Good to go!'
  elif l2_employment_type_mix_total.value > 100:
    l2_employment_type_mix_user_message.value = 'Employment types total over 100% Please correct mix.'
  else:
    l2_employment_type_mix_user_message.value = 'Employment types total under 100% Please correct mix.'

l2_employment_type_mix_on_value_change({'old': 0, 'new': 0})

for i in l2_employment_type_mix_body.children:
  i.observe(l2_employment_type_mix_on_value_change, names='value')

l2_employment_type_mix.children = [l2_employment_type_mix_header] + [l2_employment_type_mix_body]
l2_employment_type_mix

Box(children=(Box(children=(Label(value='Emploment Type Mix (percentage)', layout=Layout(display='flex', justi…

In [111]:
l1_tab        = widgets.Tab()
l1_tab_titles = ['General', 'Divisions', 'Teams'] 
[l1_tab.set_title(i, title) for i, title in enumerate(l1_tab_titles)]

l1_tab.children = [l2_employment_type_mix, l2_employment_type_mix, l2_employment_type_mix]

l1_tab